# tools for planning observations

In [ ]:
__author__ = 'Pascal Louis <plouis35@gmail.com>'
__version__ = '0.1 alpha'
__keywords__ = ['astronomy', 'astropy', 'astroquery', 'matplotlib']


In [15]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.visualization import astropy_mpl_style, quantity_support
import astropy.units as u
from astroplan.plots import plot_sky
from astroplan.plots import plot_airmass
from astropy.coordinates import AltAz, EarthLocation, SkyCoord
from astropy.time import Time
from astropy.coordinates import get_sun
from astropy.coordinates import get_body

plt.style.use(astropy_mpl_style)
quantity_support();

#%matplotlib widget


## Configuration

In [16]:
# Pises
obs_loc = EarthLocation(lat=44.0*u.deg, lon=-3.5*u.deg, height=1300*u.m)

### Exoplanets
targets = [
    'WASP-14',
    'WASP-12',                      
    'HAT-P-22',    
    'HD147506',        # HAT-P-2,                 
    'XO-1',                       
    'HD189733',                     

### EB
    'HD 112409',            # V945 Cen - bright hot short-period binary in a triple system
    'V0432 Aur',            # HD 37071 -- Spectroscopic Binary
       'WW Aur',            # 
    'V0781 Tau',    
     'CN Lyn',     
    'BF Lyn',   
    'UV Leo',    
    'UW LMi', 
     'HD93148',   
    'DV Boo',       
#    'i Boo',  
        '44 Boo', #                        225.94292 47.65417  44 I Boo                        spectro binary - i Boo -- Eclipsing Binary - H 3.12

##    'HD23642', #                         56.87250  24.28833  HD23642                         V* V1229 Tau -- Spectroscopic Binary - V 6.81
    'V0916 Her',   
##    'Gaia DR3 2210596444367488384', 
##   'V0505 Per', 
    'Sirius',
        'HD23642', #                         56.87250  24.28833  HD23642                         V* V1229 Tau -- Spectroscopic Binary - V 6.81
    'T CrB', #                           239.87583 25.92028  T CrB                           V* T CrB -- Symbiotic Star - V 10.247
    'GJ 3470',

]

"""
### OHP 2022 session
targets = [
#    'PN K  4-4', #                       281.04375 6.78194   PN K  4-4                       symb - no spectrum - V10.4
#    'Iot Lyr', #                         286.82542 36.10028  Iot Lyr                         iot Lyr -- Be Star - V 5.253
#    'CH Cyg', #                          291.13792 50.24139  CH Cyg                          symb - done
#    'RR Lyr', #                          291.36625 42.78444  RR Lyr                          V* RR Lyr -- RR Lyrae Variable - R 7.6
    '12 Vul', #                          297.76708 22.61000  12 Vul                          12 Vul -- Be Star - V 4.96
#    'SS 443', #                          297.95167 -5.81389  SS 443                          micro quasar - V 11.99
#    'PN K  4-39', #                      298.27417 23.22667  PN K  4-39                      symb - no spectrum - V 8.8
    '25 Cyg', #                          299.98000 37.04278  25 Cyg                          25 Cyg -- Be Star - V 5.19
    'QR Vul', #                          303.81625 25.59194  QR Vul                          Double or Multiple Star - V 4.75
    'P Cyg', #                           304.44667 38.03306  P Cyg                           P Cyg -- Blue Supergiant - V 4.82
#    '28 Vul', #                          309.63292 24.11611  28 Vul                          28 Vul -- Star - V 5.049
#    'BW Vul', #                          313.59333 28.52194  BW Vul                          V* BW Vul -- beta Cep Variable - V 6.54
#    'AG Peg', #                          327.75833 12.62556  AG Peg                          symb - done - V 8-9
#    'Z And', #                           353.41667 48.81833  Z And                           symb - done -
#    'R Aqr', #                           355.95625 -15.28444 R Aqr                           symb - done - V 6-10
#    'EG And', #                          11.15500  40.67944  EG And                          symb - done - V 7.0
#    'AX Per', #                          24.09458  54.25056  AX Per                          symb - done - V 10.5
#    'Mira', #                            34.83667  -2.97694  HR681                           symb - done - V 2.0
#    'UV Aur', #                          80.45375  32.51111  UV Aur                          symb - done - V 7.5
#    'SS Lep', #                          91.24625  -16.48444 SS Lep                          symb - done - V 4.9
    'T CrB', #                           239.87583 25.92028  T CrB                           V* T CrB -- Symbiotic Star - V 10.247
]
"""

obs_time = '2024-04-10'      # local time
utcoffset = +2*u.hour


from IPython.display import display
import ipywidgets as widgets

target_selected = widgets.Dropdown(options=targets, description='Target:')
display(target_selected)


Dropdown(description='Target:', options=('WASP-14', 'WASP-12', 'HAT-P-22', 'HD147506', 'XO-1', 'HD189733', 'HD…

## Show sky position

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astroplan import Observer
from astroplan.plots import plot_sky
from astroplan import FixedTarget

### get coming night times
midnight = Time(obs_time + ' 00:00:00') - utcoffset
delta_midnight = np.linspace(-12, 12, 1000)*u.hour
times_to_tomorow = midnight + delta_midnight
frame_to_tomorow = AltAz(obstime=times_to_tomorow, location=obs_loc)

### get SUN position
sunaltazs_to_tomorow = get_sun(times_to_tomorow).transform_to(frame_to_tomorow)

### get MOON position
moon_to_tomorow = get_body("moon", times_to_tomorow)
moonaltazs_to_tomorow= moon_to_tomorow.transform_to(frame_to_tomorow)

### get TARGETS position
targets = [target_selected.value]
targets_to_tomorow = [ SkyCoord.from_name(target) for target in targets ]
targets_altazs_to_tomorow = [t.transform_to(frame_to_tomorow) for t in targets_to_tomorow ]

right_panel = widgets.Output(layout={'width': '50%', 'height': '550px'})
with right_panel:
    #plt.figure(figsize=(8,6))
    plt.plot(delta_midnight, sunaltazs_to_tomorow.alt, color='orange', ls='--', label='Sun')
    plt.plot(delta_midnight, moonaltazs_to_tomorow.alt, color=[0.75]*3, ls='--', label='Moon')
    for tt,name in zip(targets_altazs_to_tomorow, targets):
        plt.scatter(delta_midnight, tt.alt, c=tt.az.value, label=name, lw=0, s=8, ls='dotted', cmap='viridis')
        #plt.plot(delta_midnight, tt.alt, label=name) #, cmap='viridis')
    plt.fill_between(delta_midnight, 0*u.deg, 90*u.deg, sunaltazs_to_tomorow.alt < -0*u.deg, color='0.5', zorder=0)
    plt.fill_between(delta_midnight, 0*u.deg, 90*u.deg, sunaltazs_to_tomorow.alt < -18*u.deg, color='k', zorder=0)
    plt.colorbar().set_label('Azimuth [deg]')
    plt.legend(loc='upper left')
    plt.xticks((np.arange(13)*2-12)*u.hour)
    plt.ylim(0*u.deg, 90*u.deg)
    plt.xlabel('Hours from EDT Midnight')
    plt.ylabel('Altitude [deg]')
    plt.xlim(-7*u.hour, 10*u.hour)
    plt.show()

left_panel = widgets.Output(layout={'width': '50%', 'height': '550px'})
with left_panel:
    observe_time = midnight + np.linspace(-4, 6, 10)*u.hour # - + np.linspace(-12, 12, 40)*u.hour
    observer = Observer(location=obs_loc)
    
    targets = [target_selected.value]
    for name in targets:
        plot_sky(target = FixedTarget.from_name(name), observer = observer, time = observe_time, grid=True) #, north_to_east_ccw = True)
    
    plt.legend(loc='center left', bbox_to_anchor=(1.25, 0.5))
    plt.show()

display(widgets.HBox([right_panel, left_panel]))


## Show SIMBAD infos

In [12]:
from IPython.display import IFrame

targets = target_selected.value
url = (f'https://simbad.cds.unistra.fr/simbad/sim-basic?Ident={targets}&submit=SIMBAD+search')
IFrame(url, width='100%', height=800)